## お手本


カラム：
- S000: 募集中のjob数
- lat_w: 目標地点の経度：　小数点第６桁
- lng_w: 目標地点の緯度：　小数点第６桁
- lat_h: 出発地点の経度：　小数点第６桁
- lng_h: 出発地点の緯度：　小数点第６桁

以下のCSVを作成する。
078kobeの後で、カラム名を変更する。

- S000: number of people
- lat_w: 目標地点（避難所）の経度
- lng_w: 目標地点（避難所）の緯度
- lat_h: 出発地点（災害発生時の場所）の経度
- lng_h: 出発地点（災害発生時の場所）の緯度
- dis: (一番近い)避難所までの距離(km)


In [79]:
import pandas as pd
import random
import sympy
import geopandas as gpd

from geopy.distance import geodesic
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
df = pd.read_csv('arc.csv')
df.head(3)

,S000,w_geocode,h_geocode,lat_w,lng_w,geoid10,lat_h,lng_h
0,2,60014226001053,60014004001009,37.871299,-122.247111,60014004001009,37.849728,-122.253480
1,2,60750615001027,60014004001009,37.788996,-122.399133,60014004001009,37.849728,-122.253480
2,3,60014011004019,60014011003009,37.834784,-122.263400,60014011003009,37.825329,-122.263653


### まずは、ポートアイランドの範囲でやってみる。

手順：
- 緯度経度の範囲を4角形で指定する。（対角線の2点の緯度経度を調べてハードコーディング）
- その中からランダムに、住民の人数分/2の位置情報を生成する。
- 地域の避難所に一番近い避難所を選ぶ（with 三平方の定理）

仮定：
- 住民の大体半分くらいが、避難する。


![imag](./assets/kobe_chuo_population.jpg)

参考：

In [109]:
# ポートアイランドの範囲を決める。

#lat_up_portisland = 34.679008
#lng_up_portisland = 135.203112
lat_up_portisland = 34.682858
lng_up_portisland = 135.199984

#lat_down_portisland = 34.654117
#lng_down_portisland = 135.228847
lat_down_portisland = 34.650632
lng_down_portisland = 135.244975


population_kobe_chuo_portisland = 15326

df_evacuation_sites = pd.read_csv('assets/神戸市避難所一覧.csv')
df_evacuation_portisland = df_evacuation_sites[df_evacuation_sites['区'] == '中央_港島']

evacuation_portisland_lng_list = [135.2140986, 135.2126072, 135.2101714, 135.210735, 135.2182844, 135.2039939]
evacuation_portisland_lat_list = [34.6696569, 34.66373, 34.6639883, 34.6631078, 34.6659964, 34.6694625]


In [110]:
df_evacuation_portisland

,番号,区,緊急避難場所の利用,避難所の利用,名称,住所,電話番号,郵便番号,土砂災害,洪水,津波,備考,世界_10進_X,世界_10進_Y
92,93,中央_港島,○,○,義務教育学校港島学園,神戸市中央区港島中町3-2-2,"078-302-1661,1771",650-0046,－,－,○,建物に被害が無ければ、原則自宅避難,135.214099,34.669657
93,94,中央_港島,○,○,神戸国際展示場,神戸市中央区港島中町6-11-1,078-302-1020,650-0046,－,－,○,建物に被害が無ければ、原則自宅避難,135.212607,34.663730
94,95,中央_港島,○,○,ワールド記念ホール,神戸市中央区港島中町6-12-2,078-302-8781,650-0046,－,－,○,建物に被害が無ければ、原則自宅避難,135.210171,34.663988
95,96,中央_港島,○,○,ポートアイランドスポーツセンター,神戸市中央区港島中町6-12-1,078-302-1031,650-0046,－,－,○,建物に被害が無ければ、原則自宅避難,135.210735,34.663108
96,97,中央_港島,○,○,バンドー神戸青少年科学館,神戸市中央区港島中町7-7-6,078-302-5177,650-0046,－,－,○,建物に被害が無ければ、原則自宅避難,135.218284,34.665996
97,98,中央_港島,○,○,神戸学院大学,神戸市中央区港島1-1-3,078-974-1551,650-0045,－,－,○,建物に被害が無ければ、原則自宅避難,135.203994,34.669463


In [111]:
gdf_hyogo_kobe_nada = gpd.read_file("./assets/border/Hyogo/Nada/r2kb28102.shp")
pisland_1_geo= gdf_hyogo_kobe_nada[gdf_hyogo_kobe_nada['S_NAME']=='港島一丁目']
#x,y = pisland_1_geo.values[0].exterior.xy

In [112]:
pisland_1_geo['S_NAME'].unique()

array([], dtype=object)

In [113]:
def random_float_between2(start, end):
    v = random.uniform(start, end)
    return v


def random_int_between2(start, end):
    v = random.randint(start, end)
    return v



# 2つの線の交点を算出する
def prog(x1, y1, x2, y2, x3, y3, x4, y4):
    '''
    @x1 float: 
    @y1 float: 
    @x2 float: 
    @y2 float: 
    @x3 float: 
    @y3 float: 
    @x4 float: 
    @y4 float: 
    '''
    if x1 == x2 and x3 == x4:
        x = y = np.nan
    elif x1 == x2:
        x = x1
        y = (y4 - y3) / (x4 - x3) * (x1 - x3) + y3
    elif x3 == x4:
        x = x3
        y = (y2 - y1) / (x2 - x1) * (x3 - x1) + y1
    else:
        a1 = (y2-y1)/(x2-x1)
        a3 = (y4-y3)/(x4-x3)
        if a1 == a3:
            x = y = np.nan
        else:
            x = (a1*x1-y1-a3*x3+y3)/(a1-a3)
            y = (y2-y1)/(x2-x1)*(x-x1)+y1
    return (x, y)






# 内外判定： 
gdf_hyogo_kobe_chuo = gpd.read_file("./assets/border/Hyogo/Chuo/r2kb28110.shp")
pisland_1_geo= gdf_hyogo_kobe_chuo[gdf_hyogo_kobe_chuo['S_NAME']=='港島一丁目']['geometry']
x,y = pisland_1_geo.values[0].exterior.xy

def check_in_or_out(target_x, target_y, x=x, y=y):
    '''
    @x array.array : 地域の境界x座標群
    @y array.array : 地域の境界y座標群
    @target_x float : 内外判定を行う対象のx座標
    @target_y float : 内外判定を行う対象のy座標
    '''
    cross_counter_x_right = 0
    cross_counter_x_left = 0
    cross_counter_y_upper = 0
    cross_counter_y_lower = 0


    if len(x) != len(y):
        #TODO: implement test
        print("[error]境界座標のx座標とy座標の個数が一致しません。")
        #return


    # （直線Aの）y軸と交差しているか判定
    for i in range(len(x)-1):
        tuple_x = (x[i], x[i+1])

        if (min(tuple_x) <= target_x <= max(tuple_x)):
            # y軸判定：交差している

            # 交点を算出
            inter_point = prog(target_x, 0, target_x, 180, x[i], y[i], x[i+1], y[i+1])

            if target_y <= inter_point[1]:
                cross_counter_y_upper +=1
            elif target_y > inter_point[1]:
                cross_counter_y_lower +=1
            else:
                #TODO: implement test
                print("[error] 本来引っかからない条件")

        elif (target_x < min(tuple_x)) or (max(tuple_x) < target_x):
            # y軸判定：交差していない
            pass
        else:
            #TODO: implement test
            print("[error] 本来引っかからない条件")


    # （直線Aの）x軸と交差しているか判定
    for i in range(len(y)-1):
        tuple_y = (y[i], y[i+1])

        if (min(tuple_y) <= target_y <= max(tuple_y)):
            # x軸判定：内側

            # 交点を算出
            inter_point = prog(0, target_y, 180, target_y, x[i], y[i], x[i+1], y[i+1])

            if target_x <= inter_point[0]:
                cross_counter_x_right +=1
            elif target_x > inter_point[0]:
                cross_counter_x_left +=1
            else:
                #TODO: implement test
                print("[error] 本来引っかからない条件")

        elif (target_y < min(tuple_y)) or (max(tuple_y) < target_y):
            # x軸判定：交差していない
            pass
        else:
            #TODO: implement test
            print("[error] 本来引っかからない条件")


    # 最終判定
    if (cross_counter_x_right % 2 == 1 ) & (cross_counter_x_left % 2 == 1 ) & (cross_counter_y_lower % 2 == 1 ) & (cross_counter_y_upper % 2 == 1 ):
        # 最終判定：　内側
        #print("True")
        return True
    else:
        # 最終判定：　内側
        #print("False")
        return False

In [120]:
population_kobe_chuo_portisland = 15326

number_of_evacuee_portisland = int(population_kobe_chuo_portisland/2)


df_data_chuo_portisland = pd.DataFrame(columns=['S000','lng_w','lat_w','lng_h', 'lat_h', 'dis'])


for i in range(number_of_evacuee_portisland) :

    if number_of_evacuee_portisland < 0:
        break
    
    # 被災時の緯度経度（ランダム）
    latitude = random_float_between2(lat_up_portisland, lat_down_portisland)
    longitude = random_float_between2(lng_up_portisland, lng_down_portisland)
    current_site = (latitude, longitude)
    
    
    # 内外判定：
    result_in_or_out = check_in_or_out(target_x=longitude, target_y=latitude)
    if result_in_or_out:
        pass
    elif result_in_or_out == False:
        continue
    else:
        #TODO: implement test
        print("[error] 本来引っかからない条件")
    
    
    # 人数（1~5人）
    number_p = random_int_between2(1,5)
    number_of_evacuee_portisland -= number_p
    
    
    target_dis_km = 100
    # 向かう避難所の緯度経度（一番近い避難所に行くと仮定）
    for row in df_evacuation_portisland.iterrows():

        # 避難所の場所
        lng = row[1]['世界_10進_X']
        lat = row[1]['世界_10進_Y']
        evacuation_site = (lat, lng)

        # 距離(km)を算出
        dis_km = geodesic(current_site, evacuation_site).km

        # 最短距離を更新。
        if (target_dis_km > dis_km):
            
            target_longitude = lng
            target_latitude = lat
            target_dis_km = dis_km
    
    
    new_row_df = pd.DataFrame(
        [[number_p, target_longitude, target_latitude, longitude, latitude, target_dis_km]],
        columns=['S000','lng_w','lat_w','lng_h', 'lat_h', 'dis']
    )
    
    df_data_chuo_portisland = pd.concat([df_data_chuo_portisland, new_row_df], axis=0, ignore_index=True)


In [121]:
# 避難所毎の人数を計算する。

#（避難所のlng : 避難所計人数）
site_map = {}
unique_sites = df_data_chuo_portisland['lng_w'].unique().tolist()

for lng_w in unique_sites:
    df_one_site = df_data_chuo_portisland[df_data_chuo_portisland['lng_w']==lng_w]
    sum_p = df_one_site['S000'].sum()

    site_map[str(lng_w)] = sum_p
    

new_col_data = []
for row in df_data_chuo_portisland.iterrows():
    lng_w = row[1]['lng_w']
    sum_p = site_map[str(lng_w)]
    new_col_data.append(sum_p)
    

df_data_chuo_portisland['sum_p'] = new_col_data

In [122]:
df_data_chuo_portisland

,S000,lng_w,lat_w,lng_h,lat_h,dis,sum_p
0,1,135.210171,34.663988,135.205152,34.661947,0.512798,278
1,5,135.203994,34.669463,135.207076,34.671040,0.332328,729
2,3,135.203994,34.669463,135.201901,34.669624,0.192700,729
3,4,135.203994,34.669463,135.203317,34.669116,0.073009,729
4,1,135.203994,34.669463,135.204933,34.668367,0.148944,729
...,...,...,...,...,...,...,...
402,5,135.203994,34.669463,135.203901,34.665527,0.436626,729
403,5,135.203994,34.669463,135.207033,34.670520,0.302281,729
404,5,135.203994,34.669463,135.202850,34.664514,0.558826,729
405,4,135.203994,34.669463,135.206172,34.668822,0.211895,729


In [123]:
df_data_chuo_portisland.to_csv('df_evacuee_data_kobe_chuo_portisland_v2.csv',header=True, index=False)